#### annoation 40개 이상 제거

In [1]:
from pycocotools.coco import COCO
import json
from collections import defaultdict

# 데이터셋 경로
dataset_path = './dataset'
train_data = COCO(f'{dataset_path}/train.json')

# 이미지 ID와 각 이미지에 속한 annotation 개수를 저장할 딕셔너리
image_id_to_annotation_count = defaultdict(int)

# 모든 annotation을 가져와서 이미지별로 개수 세기
for annotation in train_data.dataset['annotations']:
    image_id = annotation['image_id']
    image_id_to_annotation_count[image_id] += 1

# 30개 이하의 annotation을 가진 이미지들의 ID 수집
selected_image_ids = [image_id for image_id, count in image_id_to_annotation_count.items() if count <= 40]

# 새로운 JSON 구조 생성
new_coco_data = {
    'images': [],
    'annotations': [],
    'categories': train_data.dataset['categories']  # 카테고리 정보는 그대로 유지
}

# 선택된 이미지에 해당하는 데이터만 새로운 COCO 데이터에 추가
new_image_ids_set = set(selected_image_ids)
new_image_id_to_index = {}

for image in train_data.dataset['images']:
    if image['id'] in new_image_ids_set:
        new_image_id_to_index[image['id']] = len(new_coco_data['images']) + 1  # 새로운 ID로 매핑
        image_copy = image.copy()
        image_copy['id'] = new_image_id_to_index[image['id']]  # 이미지 ID 갱신
        new_coco_data['images'].append(image_copy)

for annotation in train_data.dataset['annotations']:
    if annotation['image_id'] in new_image_ids_set:
        annotation_copy = annotation.copy()
        annotation_copy['image_id'] = new_image_id_to_index[annotation['image_id']]  # 새로운 이미지 ID로 갱신
        new_coco_data['annotations'].append(annotation_copy)

# 새로운 JSON 파일 저장 (들여쓰기 포함)
with open(f'{dataset_path}/train_ann_len_under_40.json', 'w') as f:
    json.dump(new_coco_data, f, indent=2)

print(f"새로운 json 파일 생성 완료: {dataset_path}/train_ann_len_under_40.json")

loading annotations into memory...
Done (t=0.03s)
creating index...
index created!
새로운 json 파일 생성 완료: ./dataset/train_ann_len_under_40.json


In [27]:
import os

os.makedirs(f'{dataset_path}/labels', exist_ok=True)

for i in train_data.getImgIds():
    img_info = train_data.loadImgs(ids=i)[0]
    img_width = img_info['width']
    img_height = img_info['height']
    file_name = img_info['file_name'].replace("jpg", "txt").replace('train/', '')

    with open(f'{dataset_path}/labels/{file_name}', 'w') as f:
        for j in train_data.loadAnns(train_data.getAnnIds(imgIds=i)):
            # category_id는 0부터 시작하도록 설정
            category_id = j['category_id']# YOLO 형식은 보통 0부터 시작

            # bbox 변환: [x_min, y_min, width, height] -> [x_center, y_center, width, height]
            x_min, y_min, width, height = j['bbox']
            x_center = x_min + width / 2
            y_center = y_min + height / 2

            # 좌표를 이미지 크기로 정규화 (0~1 값으로 변환)
            x_center /= img_width
            y_center /= img_height
            width /= img_width
            height /= img_height

            # YOLO 형식으로 파일에 작성: category_id, x_center, y_center, width, height
            f.write(f"{category_id} {x_center} {y_center} {width} {height}\n")

